In [2]:
# ~ 0.5s
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib widget

gestures_train = pd.read_csv('gestures_train.csv', index_col=0)
gestures_train_2 = pd.read_csv('gestures_train_2.csv', index_col=0)

OMG_CH = [str(i) for i in range(50)]

ACC_CH = ['ACC0', 'ACC1', 'ACC2']
GYR_CH = ['GYR0', 'GYR1', 'GYR2']
# ENC_CH = ['ENC0', 'ENC1', 'ENC2', 'ENC3', 'ENC4', 'ENC5']
GLOVE_CH = ['ENC0', 'ENC1', 'ENC2', 'ENC3', 'ENC4', 'ENC5']
BUTTON_SYNC_TS_CH = ['BUTTON', 'SYNC', 'ts']

assert len(OMG_CH)+len(ACC_CH)+len(GYR_CH)+len(GLOVE_CH)+len(BUTTON_SYNC_TS_CH) == gestures_train.shape[-1]

print(f"OMG_CH: {list(OMG_CH)}")
print(f"ACC_CH: {list(ACC_CH)}")
print(f"GYR_CH: {list(GYR_CH)}")
print(f"GLOVE_CH: {list(GLOVE_CH)}")
print(f"BUTTON_SYNC_TS_CH: {list(BUTTON_SYNC_TS_CH)}")

gestures_test = pd.read_csv('gestures_test.csv', index_col=0)
gestures_test_2 = pd.read_csv('gestures_test_2.csv', index_col=0)

gestures_protocol = pd.read_csv('gestures_protocol.csv')

mask_open = (gestures_protocol[['Thumb_stretch', 'Index_stretch', 'Middle_stretch', 'Ring_stretch', 'Pinky_stretch']] == 1.0).all(axis=1)
gestures_protocol.loc[mask_open, ['Thumb', 'Index', 'Middle', 'Ring', 'Pinky']] = -0.5

gestures_train_ext = pd.merge(
    gestures_train,
    gestures_protocol,
    how='left',
    left_on='SYNC',
    right_on='epoch',
)

gestures_test_ext = pd.merge(
    gestures_test,
    gestures_protocol,
    how='left',
    left_on='SYNC',
    right_on='epoch',
)


free_movements = pd.read_csv('free_movements.csv', index_col=0)
free_movements_2 = pd.read_csv('free_movements_2.csv', index_col=0)

# Прочитаем данные 2-х пилотов
X_1 = gestures_train[OMG_CH].values
X_test_1 = gestures_test[OMG_CH].values
y_1 = gestures_train[GLOVE_CH].values
y_test_1 = gestures_test[GLOVE_CH].values

X_2 = gestures_train_2[OMG_CH].values
X_test_2 = gestures_test_2[OMG_CH].values
y_2 = gestures_train_2[GLOVE_CH].values
y_test_2 = gestures_test_2[GLOVE_CH].values

# И последовательно соединим тренировочные данные 2-х пилотов
X_pilots = np.vstack([X_1, X_2])
y_pilots = np.vstack([y_1, y_2])
X_test_pilots = np.vstack([X_test_1, X_test_2])
y_test_pilots = np.vstack([y_test_1, y_test_2])

# Прочтём свободные жесты обоих пилотов
free_movements = pd.read_csv('free_movements.csv', index_col=0)
free_movements_2 = pd.read_csv('free_movements_2.csv', index_col=0)

# И последовательно соединим их
X_free = np.vstack([free_movements[OMG_CH], free_movements_2[OMG_CH]])
y_free = np.vstack([free_movements[GLOVE_CH], free_movements_2[GLOVE_CH]])

# Соединим протокольные и свободные жесты обоих пилотов
X_all = np.vstack([X, X_free])
y_all = np.vstack([y, y_free])

OMG_CH: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49']
ACC_CH: ['ACC0', 'ACC1', 'ACC2']
GYR_CH: ['GYR0', 'GYR1', 'GYR2']
GLOVE_CH: ['ENC0', 'ENC1', 'ENC2', 'ENC3', 'ENC4', 'ENC5']
BUTTON_SYNC_TS_CH: ['BUTTON', 'SYNC', 'ts']


In [3]:
# ~ 18s
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LassoLars
from sklearn.ensemble import VotingRegressor

estimators = [
    ('dt', DecisionTreeRegressor()),('ll', LassoLars(alpha=0.1))
]
model_vr = MultiOutputRegressor(VotingRegressor(estimators=estimators, weights=[0.45, 0.55])).fit(X_all, y_all)

In [4]:
# ~ 22s
import xgboost as xgb

# параметры оптимизированы optun'ой
params = {'learning_rate': 0.06329973864656831,
 'max_depth': 6,
 'subsample': 0.9814063371832862,
 'colsample_bytree': 0.41087811860602663,
 'min_child_weight': 8}

model_xgb = xgb.XGBRegressor(**params)
model_xgb.fit(X_all, y_all)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.41087811860602663, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.06329973864656831,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=8, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [5]:
# ~ 7s
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb

# параметры оптимизированы optun'ой
params = {'learning_rate': 0.07118878873086158,
 'num_leaves': 78,
 'max_depth': 159,
 'min_data_in_leaf': 1423}

model_lgb = MultiOutputRegressor(lgb.LGBMRegressor(**params))
model_lgb.fit(X_all, y_all)

[LightGBM] [Warning] min_data_in_leaf is set=1423, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1423


MultiOutputRegressor(estimator=LGBMRegressor(learning_rate=0.07118878873086158,
                                             max_depth=159,
                                             min_data_in_leaf=1423,
                                             num_leaves=78))

In [6]:
# ~ 0.1s
df_sim = gestures_test.iloc[:10000] # первый пилот, протокольные жесты
#df_sim = free_movements.iloc[:2000] # первый пилот свободные жесты
#df_sim = gestures_test_2.iloc[:10000] # первый пилот, протокольные жесты
#df_sim = free_movements_2.iloc[:2000] # второй пилот свободные жесты
print(df_sim.shape)

(7244, 65)


In [7]:
# ~ 0.1s
crc_table = [
    54, 181, 83, 241, 89, 16, 164, 217, 34, 169, 220, 160, 11, 252,
    111, 241, 33, 70, 99, 240, 234, 215, 60, 206, 68, 126, 152, 81,
    113, 187, 14, 21, 164, 172, 251, 16, 248, 215, 236, 90, 49, 53,
    179, 156, 101, 55, 65, 130, 161, 22, 218, 79, 24, 168, 152, 205,
    115, 141, 23, 86, 141, 58, 122, 215, 252, 48, 69, 115, 138, 66,
    88, 37, 63, 104, 176, 46, 139, 246, 222, 184, 103, 92, 154, 174,
    97, 141, 195, 166, 227, 150, 140, 48, 121, 243, 13, 131, 210, 199,
    45, 75, 180, 104, 97, 82, 251, 90, 132, 111, 229, 175, 146, 216,
    153, 86, 166, 33, 184, 100, 225, 248, 186, 54, 89, 39, 2, 214, 2,
    114, 197, 6, 35, 188, 245, 64, 220, 37, 123, 132, 190, 60, 189, 53,
    215, 185, 238, 145, 99, 226, 79, 54, 102, 118, 210, 116, 51, 247,
    0, 191, 42, 45, 2, 132, 106, 52, 63, 159, 229, 157, 78, 165, 50,
    18, 108, 193, 166, 253, 3, 243, 126, 111, 199, 152, 36, 114, 147,
    57, 87, 14, 16, 160, 128, 97, 189, 51, 115, 142, 8, 70, 71, 55, 42,
    193, 65, 207, 122, 158, 26, 21, 72, 139, 33, 230, 230, 116, 134, 5,
    213, 165, 107, 41, 134, 219, 190, 26, 29, 136, 174, 190, 108, 185, 172,
    137, 239, 164, 208, 207, 206, 98, 207, 12, 0, 174, 64, 20, 90, 49, 12,
    67, 112, 109, 78, 114, 165, 244, 183, 121
];
    
def get_crc(data, count):
    result = 0x00
    l = count
    while (count>0):   
        result = crc_table[result ^ data[l-count]]
        count -= 1;
    return result;

def drv_abs_one(ser, control):
    if ser is None:
        return
    pack = (np.array([80, 0x01, 0xBB, 6] + [c for c in control] + [0], dtype='uint8'))
    pack[-1] = get_crc(pack[1:], 9)
    return (pack, ser.write(bytearray(pack)))

## Что сделано:

* Подготовлены к работе 3 модели (VotingRegressor, XGBRegressor, LGBMRegressor)
* В качестве предпрцессинга - уникальное ничего
* Постпроцессинг:
* * дискретизация таргетов на заданное количество диапазонов (по умолчанию 20 диапазон, по 5 единиц в каждом) 
* * сглаживатель пиков по предыдущим показаниям с шагом 3 и 4 (по умолчанию шаг 3) даёт задержку между предсказанием и выводом на протез в 1 временной шаг
* * 

In [8]:
# ~ 0.1s
import time
import serial
from IPython.display import clear_output

from sklearn.metrics import mean_squared_error as mse
import collections # нужно для deque (очередь нужна чтобы хранить пред-предыдущее предсказание)

def MSE(y, y_pred):
    """_получение среднеквадратичной ошибки для каждого таргета_

    """
    ENC0 = mse(y[:,0], y_pred[:,0])
    ENC1 = mse(y[:,1], y_pred[:,1])
    ENC2 = mse(y[:,2], y_pred[:,2])
    ENC3 = mse(y[:,3], y_pred[:,3])
    ENC4 = mse(y[:,4], y_pred[:,4])
    ENC5 = mse(y[:,5], y_pred[:,5])
    return pd.Series([ENC0, ENC1, ENC2, ENC3, ENC4, ENC5])

def preprocessing(x):
    # x - sample vector
    y = x
    return y

def inference(x):
    y = model_xgb.predict([x])[0]
    # y = model_lgb.predict([x])[0] # модель на light_boost
    # y = model_vr.predict([x])[0] # модель на xgb_boost
    return y

def postprocessing(array, step=5):
    """_дискретизация выходных сигналов по 100/step количеству уровней
        по умолчанию step=10 -> 10 уровней сигналов_
    """
    array[array < 0] = 0
    array = np.round(array / step, 0).astype(int) * step
    return array

def commands(dq):
    """_сглаживатель пиков по 2-м предыдущим шагам_
    """
    if len(dq) < 2:
        return np.zeros(6)
    else:
        if (dq[-1] == dq[0]).any():
            dq[1][dq[-1] == dq[0]] = dq[0][dq[-1] == dq[0]]
    return dq[-1]

def commands_1(dq):
    """_сглаживатель пиков по 3-м предыдущим шагам_
    """
    if len(dq) < 3:
        return np.zeros(6)
    else:
        if (dq[-1] == dq[0]).any():
            dq[1][dq[-1] == dq[0]] = dq[0][dq[-1] == dq[0]]
            dq[2][dq[-1] == dq[0]] = dq[0][dq[-1] == dq[0]]
    return dq[0]

def commands_0(x):
    if len(x) < 2:
        return np.zeros(6)
    else:
        return x[-1]

In [9]:
TIMEOUT = 0.33
DEBUG = False
    
i = 0
ts_old = time.time()
ts_diff = 0;

dq = collections.deque(maxlen=3)

y_dct = {
    'omg_sample':[],
    'enc_sample':[],
    'sample_preprocessed':[],
    
    'y_predicted':[],
    'y_postprocessed':[],
    'y_commands':[],
}
while True:    
    
    # [Data reading]
    ts_start = time.time()
    
    try:
        # [Sim data]
        if i < len(df_sim):
            sample = df_sim.values[i]
        else:
            break
        # [/Sim data]
        [omg_sample, acc_sample, enc_sample, [button, sync, ts]] = np.array_split(sample, [50, 56, 62])
        
    except Exception as e:
        print(e)
        
    # [/Data Reading]
        
    # [Data preprocessing]
    sample_preprocessed = preprocessing(omg_sample)
    # [/Data preprocessing]
    
    # [Inference]
    y_predicted         = inference(sample_preprocessed)
    # [/Inference]
    
    # [Inference Postprocessing]
    y_postprocessed     = postprocessing(y_predicted)
    # [/Inference Postprocessing]
       
    # [Commands composition]
    y_commands          = commands(dq)
    # [/Commands composition]
    
    # [Commands sending]
    # NO COMMANDS SENDING IN SIMULATION
    # [/Commands sending]
    
    # [Data logging]
    y_dct['omg_sample'].append(omg_sample)
    y_dct['enc_sample'].append(enc_sample)
    y_dct['sample_preprocessed'].append(sample_preprocessed)
    y_dct['y_predicted'].append(y_predicted)
    y_dct['y_postprocessed'].append(y_postprocessed)
    y_dct['y_commands'].append(y_commands)
    # [/Data logging]

    dq.append(y_postprocessed)
    
    
    if DEBUG:
        clear_output(wait=True)

        # sanity check: Sizes of SAMPLE=65, OMG=50, ACC=6, ENCODERS=6
        print(f'SAMPLE SIZE: {len(sample)}, OMG: {len(omg_sample)}, ACC: {len(acc_sample)}, ENCODERS: {len(enc_sample)}')
        print(f'BUTTON: {button}, SYNC: {sync}, TS: {ts}')
        print(y_commands)
    
    ts_diff = time.time() - ts_start
    assert(ts_diff<TIMEOUT), 'Calculation cycle takes more than TIMEOUT, halting...'
    ts_old = ts_start
    i += 1 

In [10]:
for key, val in y_dct.items():
    # print(f"len({key}) = {len(y_dct[key])}")
    y_dct[key] = np.stack(val)
    print(f"{key}.shape = {y_dct[key].shape}")
    
metrics_test = {col : mse(y_dct['enc_sample'][:,col], y_dct['y_commands'][:,col]) for col in np.arange(6)}

metrics_test = pd.Series(metrics_test)
metrics_test

omg_sample.shape = (7244, 50)
enc_sample.shape = (7244, 6)
sample_preprocessed.shape = (7244, 50)
y_predicted.shape = (7244, 6)
y_postprocessed.shape = (7244, 6)
y_commands.shape = (7244, 6)


0    112.614716
1    129.751518
2    135.725980
3    127.193263
4    359.715351
5      0.000000
dtype: float64

### Online (prosthesis or virtual hand)

In [11]:
TIMEOUT = 0.033
DEBUG = True

dq = collections.deque(maxlen=4)

ser = None
# ser_port = None
ser_port = '/dev/cu.usbmodem3498365F31351'

if ser_port is not None:
    ser = serial.Serial(port=ser_port, timeout=2*TIMEOUT)
    ser.write('T1#\r\n'.encode('utf-8')) # T1 for Timestamp activate
    ser.write('M2#\r\n'.encode('utf-8')) # M2 for Mode == 2 = send samples
    ser.write('S2#\r\n'.encode('utf-8')) # SYNC to 2 for sanity check

    # flush buffers
    ser.reset_input_buffer()
    ser.read()
    
    i = 0;
    while(ser.in_waiting):
        print(f'Flushing buffers {i}: {ser.in_waiting}', end='    \r')
        ser.read_all()
        time.sleep(0.005)
        i+=1;
    ser.readline()
    ser.readline()
    
i = 0
ts_old = time.time()
ts_diff = 0;

y_previous = None
while True:    
    
    # [Data reading]
    s = ser.readline()
    ts_start = time.time()
    
    try:
        sample = np.array(s.decode('UTF-8')\
                           .replace('\r\n', "")\
                           .split(' ')
                         ).astype(int)
        [omg_sample, acc_sample, enc_sample, [button, sync, ts]] = np.array_split(sample, [50, 56, 62])
        
    except Exception as e:
        print(e)
        
    # [/Data Reading]
        
    # [Data preprocessing]
    sample_preprocessed = preprocessing(omg_sample)
    # [/Data preprocessing]
    
    # [Inference]
    y_predicted         = inference(sample_preprocessed)
    # [/Inference]
    
    # [Inference Postprocessing]
    y_postprocessed     = postprocessing(y_predicted)
    # [/Inference Postprocessing]
    
    # [Commands composition]
    y_commands          = commands_1(dq)
    # [/Commands composition]
    
    # [Commands sending]
    pack, _ = drv_abs_one(ser, list(y_commands)); # + [0]
    # [/Commands sending]
    
    dq.append(y_postprocessed)
    
    if DEBUG:
        clear_output(wait=True)

        # sanity check: iteration should increase monotonically, TIMEDIFF approximately 32-34 ms, CYCLETIME < TIMEOUT, WAITING should be == 0
        print(f'ITERATION:\t{i}\tTIMEDIFF:\t{(ts_start - ts_old)*1000: .0f}\tCYCLETIME:\t{ts_diff*1000:.0f}\tWAITING:\t{ser.in_waiting}')
        print('INPUT:\n', s)

        # sanity check: Sizes of SAMPLE=65, OMG=50, ACC=6, ENCODERS=6
        print(f'SAMPLE SIZE: {len(sample)}, OMG: {len(omg_sample)}, ACC: {len(acc_sample)}, ENCODERS: {len(enc_sample)}')
        print(f'BUTTON: {button}, SYNC: {sync}, TS: {ts}')
        print(y_commands)
        print(pack)
    
    ts_diff = time.time() - ts_start
    assert(ts_diff<TIMEOUT), 'Calculation cycle takes more than TIMEOUT, halting...'
    ts_old = ts_start
    i += 1 

SerialException: [Errno 2] could not open port /dev/cu.usbmodem3498365F31351: [Errno 2] No such file or directory: '/dev/cu.usbmodem3498365F31351'